In [219]:
from dataclasses import dataclass, field
from typing import Tuple
from parse import parse
from collections import Counter
import string

import black
import jupyter_black


jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [32]:
# Day 1: No Time for a Taxicab
@dataclass
class Position:
    heading: int
    x: int
    y: int
    visited: set = field(default_factory=set)
    double_visit_distance: int = 0

    # NESW = 0, 1, 2, 3
    HEADINGS = ((0, 1), (1, 0), (0, -1), (-1, 0))

    def forward(self, steps: int):
        "Move forward `steps` given current heading"
        for _ in range(steps):
            self.x = self.x + self.HEADINGS[self.heading][0]
            self.y = self.y + self.HEADINGS[self.heading][1]

            if not self.double_visit_distance:
                if (self.x, self.y) in self.visited:
                    self.double_visit_distance = abs(self.x) + abs(self.y)
                self.visited.add((self.x, self.y))

    def turn_right(self):
        self.heading = (self.heading + 1) % 4

    def turn_left(self):
        self.heading = (self.heading - 1) % 4


directions = [(dir[0], int(dir[1:])) for dir in open("2016/1.txt").read().split(", ")]

pos = Position(0, 0, 0)
for turn, steps in directions:
    if turn == "R":
        pos.turn_right()
    elif turn == "L":
        pos.turn_left()
    else:
        raise ValueError(dir)
    pos.forward(steps)

print(f"Part 1: {abs(pos.x) + abs(pos.y)}")  # 236
print(f"Part 2: {pos.double_visit_distance}")  # 182

Part 1: 236
Part 2: 182


In [21]:
# Day 2: Bathroom Security
@dataclass
class Keypad:
    """
    Implements a keypad where you can move up, down, right, and left. You can't move past
    an edge. Represents current value as a string of this object.

    Keypad looks like this:

    1 2 3
    4 5 6
    7 8 9
    """

    number: int = 5

    def up(self):
        if self.number not in (1, 2, 3):
            self.number -= 3

    def down(self):
        if self.number not in (7, 8, 9):
            self.number += 3

    def left(self):
        if self.number not in (1, 4, 7):
            self.number -= 1

    def right(self):
        if self.number not in (3, 6, 9):
            self.number += 1

    def __str__(self):
        return str(self.number)


@dataclass
class BathroomKeypad:
    """
    Keypad looks like below.

        1
      2 3 4
    5 6 7 8 9
      A B C
        D
    """

    number: int = 5
    # Internal representation of keypad
    #       1
    #    2  3  4
    # 5  6  7  8  9
    #   10 11 12
    #      13

    def up(self):
        if self.number not in (5, 2, 1, 4, 9):
            if self.number in (3, 13):
                self.number -= 2
            elif self.number in (6, 7, 8, 10, 11, 12):
                self.number -= 4

    def down(self):
        if self.number not in (5, 10, 13, 12, 9):
            if self.number in (1, 11):
                self.number += 2
            else:
                self.number += 4

    def left(self):
        if self.number not in (1, 2, 5, 10, 13):
            self.number -= 1

    def right(self):
        if self.number not in (1, 4, 9, 12, 13):
            self.number += 1

    def __str__(self) -> str:
        return hex(self.number)[-1]


lines = open("2016/2.txt").readlines()

a = b = ""
keypad = Keypad()
bathroom = BathroomKeypad()
for line in lines:
    for move in line:
        match move:
            case "U":
                keypad.up()
                bathroom.up()
            case "D":
                keypad.down()
                bathroom.down()
            case "L":
                keypad.left()
                bathroom.left()
            case "R":
                keypad.right()
                bathroom.right()
    a += str(keypad)
    b += str(bathroom)
print(f"Part 1: {a}")
print(f"Part 1: {b}")

Part 1: 52981
Part 1: 74cd2


In [220]:
# Day 3: Squares With Three Sides
def possible(sides: Tuple[int, int, int]) -> bool:
    "Returns true if this combination is possible."
    a, b, c = sorted(sides)
    return a + b > c


lines = open("2016/3.txt").readlines()
lines = [line.strip().split() for line in lines]

sides = [tuple(int(side) for side in line) for line in lines]
print(f"Part 1: {sum(possible(side) for side in sides)}")  # 869

sides = [side for i in range(0, len(sides), 3) for side in zip(*sides[i : i + 3])]
print(f"Part 2: {sum(possible(side) for side in sides)}")  # 1544

Part 1: 869
Part 2: 1544


In [230]:
# Day 4: Security Through Obscurity
def real_room(name: str, checksum: str) -> bool:
    cnt = Counter(sorted(name.replace("-", "")))
    chk = "".join([kv[0] for kv in cnt.most_common(5)])
    return chk == checksum


def decrypt_room(room: str, steps: int) -> str:
    def rotate_character(char: str, steps: int) -> str:
        # Convert to base 36. a=10, b=11, ... z=35
        number = int(char, 36) - 10
        number = (number + steps) % 26
        return string.ascii_lowercase[number]

    decrypted = []
    for char in room:
        if char != "-":
            decrypted.append(rotate_character(char, steps))
        else:
            decrypted.append(" ")
    return "".join(decrypted)


lines = open("2016/4.txt").readlines()
rooms = [parse("{}-{:d}[{}]", line.strip()) for line in lines]
id_sum = 0
for name, sector, checksum in rooms:
    if real_room(name, checksum):
        id_sum += sector
        # Part 2: Sector ID of North Pole objects
        if "north" in decrypt_room(name, sector):
            print(f"Part 2: {sector}: {decrypt_room(name, sector)}")  # 482
print(f"Part 1: {id_sum}")

Part 2: 482: northpole object storage
Part 1: 361724
